In [1]:
import numpy as np
import pandas as pd
import fancyimpute
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

Using TensorFlow backend.


In [2]:
df = pd.read_csv("data/full2.csv")
# Examine head of data
df.head()

,Year,Name,Position,Round,Pick,College,Conference,Games,Seasons,Height,...,PR.TD,KRs,KR.Yds,KR.TD,XPA,XP.,FGA,FG.,Punts,Punt.Avg
0,2008,Husain Abdullah,FS,0,0,Small,Pac-10,45,4,72,...,0,0,0,0,0,0.0,0,0.0,0,0.0
1,2008,Jamar Adams,SS,0,0,Michigan,Big Ten,38,3,74,...,0,0,0,0,0,0.0,0,0.0,0,0.0
2,2008,Xavier Adibi,OLB,4,118,Virginia Tech,ACC,47,4,74,...,0,0,0,0,0,0.0,0,0.0,0,0.0
3,2008,Erik Ainge,QB,5,162,Tennessee,SEC,43,4,77,...,0,0,0,0,0,0.0,0,0.0,0,0.0
4,2008,Branden Albert,OG,1,15,Virginia,ACC,0,0,78,...,0,0,0,0,0,0.0,0,0.0,0,0.0


In [3]:
# We know that combine data is missing, so let's impute it.
imp = df[["Position", "Height", "Weight", "X40.Yard", "Bench", "Broad.Jump", "Shuttle", "X3.Cone", "Vertical"]]
imp=imp.replace({0:np.nan})
number = LabelEncoder()
imp["Position"] = number.fit_transform(imp["Position"])
imp=fancyimpute.KNN(k=5).complete(imp)
imp = pd.DataFrame(imp)
imp.columns = ["Position", "Height", "Weight", "X40.Yard", "Bench", "Broad.Jump", "Shuttle", "X3.Cone", "Vertical"]
imp = imp.round({"X40.Yard": 2, "Bench": 0, "Broad.Jump": 0, "X3.Cone": 2, "Shuttle": 2})
imp["Vertical"] = round(imp["Vertical"] * 2) / 2
df[["Height", "Weight", "X40.Yard", "Bench", "Broad.Jump", "Shuttle", "X3.Cone", "Vertical"]] = imp.drop("Position", axis=1)

Imputing row 1/3151 with 1 missing, elapsed time: 1.903
Imputing row 101/3151 with 0 missing, elapsed time: 1.911
Imputing row 201/3151 with 0 missing, elapsed time: 1.917
Imputing row 301/3151 with 0 missing, elapsed time: 1.923
Imputing row 401/3151 with 1 missing, elapsed time: 1.929
Imputing row 501/3151 with 0 missing, elapsed time: 1.935
Imputing row 601/3151 with 0 missing, elapsed time: 1.942
Imputing row 701/3151 with 0 missing, elapsed time: 1.947
Imputing row 801/3151 with 5 missing, elapsed time: 1.954
Imputing row 901/3151 with 0 missing, elapsed time: 1.960
Imputing row 1001/3151 with 0 missing, elapsed time: 1.965
Imputing row 1101/3151 with 0 missing, elapsed time: 1.969
Imputing row 1201/3151 with 1 missing, elapsed time: 1.974
Imputing row 1301/3151 with 0 missing, elapsed time: 1.980
Imputing row 1401/3151 with 5 missing, elapsed time: 1.986
Imputing row 1501/3151 with 1 missing, elapsed time: 1.992
Imputing row 1601/3151 with 3 missing, elapsed time: 2.002
Imputing 

In [4]:
# Impute missing games and season values
imp = df[["Position", "Games", "Seasons"]]
imp=imp.replace({0:np.nan})
imp["Position"] = number.fit_transform(imp["Position"])
imp=fancyimpute.KNN(k=10).complete(imp)
imp = pd.DataFrame(imp)
imp = imp.round(0)
df[["Games", "Seasons"]] = imp.drop(0, 1)

Imputing row 1/3151 with 0 missing, elapsed time: 1.367
Imputing row 101/3151 with 0 missing, elapsed time: 1.368
Imputing row 201/3151 with 2 missing, elapsed time: 1.369
Imputing row 301/3151 with 0 missing, elapsed time: 1.370
Imputing row 401/3151 with 0 missing, elapsed time: 1.372
Imputing row 501/3151 with 0 missing, elapsed time: 1.374
Imputing row 601/3151 with 2 missing, elapsed time: 1.375
Imputing row 701/3151 with 0 missing, elapsed time: 1.376
Imputing row 801/3151 with 0 missing, elapsed time: 1.378
Imputing row 901/3151 with 0 missing, elapsed time: 1.380
Imputing row 1001/3151 with 0 missing, elapsed time: 1.381
Imputing row 1101/3151 with 0 missing, elapsed time: 1.383
Imputing row 1201/3151 with 0 missing, elapsed time: 1.384
Imputing row 1301/3151 with 0 missing, elapsed time: 1.385
Imputing row 1401/3151 with 0 missing, elapsed time: 1.387
Imputing row 1501/3151 with 0 missing, elapsed time: 1.388
Imputing row 1601/3151 with 0 missing, elapsed time: 1.389
Imputing 

# Fix Position Data

In [5]:
# Let's get some information about the positions.
df.groupby("Position").size()

Position
C        96
CB      342
DB        2
DE      246
DT      247
EDGE     19
FB       45
FS      102
ILB     153
K        46
LB        2
LS       14
NT        3
OG      125
OL        2
OLB     241
OT      175
P        60
QB      177
RB      295
S        24
SS      110
TE      166
WR      459
dtype: int64

In [6]:
df[df.Position=="DB"]

,Year,Name,Position,Round,Pick,College,Conference,Games,Seasons,Height,...,PR.TD,KRs,KR.Yds,KR.TD,XPA,XP.,FGA,FG.,Punts,Punt.Avg
2954,2018,Minkah Fitzpatrick,DB,1,11,Alabama,SEC,42.0,3.0,73.0,...,0,1,16,1,0,0.0,0,0.0,0,0.0
3039,2018,Quenton Meeks,DB,0,0,Stanford,Pac-12,36.0,3.0,74.0,...,0,0,0,0,0,0.0,0,0.0,0,0.0


In [7]:
# Fitzpatrick will play mostly as a free safety and Meeks as a CB.
df.at[2954, "Position"] = "FS"
df.at[3039, "Position"] = "CB"

In [8]:
df[df.Position=="OL"]

,Year,Name,Position,Round,Pick,College,Conference,Games,Seasons,Height,...,PR.TD,KRs,KR.Yds,KR.TD,XPA,XP.,FGA,FG.,Punts,Punt.Avg
3030,2018,Cole Madison,OL,0,0,Small,Pac-12,40.0,4.0,77.0,...,0,0,0,0,0,0.0,0,0.0,0,0.0
3073,2018,Martinas Rankin,OL,3,80,Small,SEC,41.0,4.0,76.0,...,0,0,0,0,0,0.0,0,0.0,0,0.0


In [9]:
# Madison is a guard and Rankin is a tackle.
df.at[3030, "Position"] = "OG"
df.at[3073, "Position"] = "OT"

In [10]:
df.loc[df.Position=="LB", "Position"] = "OLB"
df.loc[df.Position=="NT", "Position"] = "DT"

In [11]:
# We next need to divide free safeties and strong safeties in our 2018 data. Let's
# examine what our safeties look like in our data. Let's use a simple logistic
# regression model based on Height, Weight, Total Tackles, and Interceptions.
s = df[(df.Position=="FS") | (df.Position=="SS")]
s = s[["Position", "Height", "Weight", "Total.Tackles", "Int", "Sacks"]]
s.head()
s_predict = LogisticRegressionCV()
s_predict.fit(s.drop("Position", 1), s.Position)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [12]:
safeties = df[df.Position=="S"]
df.loc[safeties.index, "Position"] = s_predict.predict(safeties[["Height", "Weight", "Total.Tackles", "Int", "Sacks"]])

In [13]:
# Let's do the same for edge rushers, forecasting their position by height and weight differences,
# as well as total tackles
outside = df[(df.Position=="OLB") | (df.Position=="DE")]
outside = outside[["Position", "Height", "Weight", "Total.Tackles"]]
outside.head()
edge_predict = LogisticRegressionCV()
edge_predict.fit(outside.drop("Position", 1), outside.Position)
edge = df[df.Position=="EDGE"]
df.loc[edge.index, "Position"] = edge_predict.predict(edge[["Height", "Weight", "Total.Tackles"]])

# Exploratory Analysis

In [14]:
# All of our "hybrid" positions have been dealt with. Let's now examine preliminary trends.
round_info = pd.crosstab(index=df["Position"], columns=df["Round"], margins=True) 
round_info / round_info.loc["All"]

Round,0,1,2,3,4,5,6,7,All
Position,,,,,,,,,
C,0.034156,0.023669,0.028481,0.026549,0.039275,0.013699,0.035714,0.034934,0.030467
CB,0.083491,0.130178,0.136076,0.147493,0.108761,0.136986,0.095238,0.078603,0.108854
DE,0.062619,0.121302,0.094937,0.097345,0.075529,0.051370,0.083333,0.096070,0.080292
DT,0.063567,0.100592,0.104430,0.109145,0.063444,0.065068,0.075397,0.087336,0.079340
FB,0.018027,0.000000,0.000000,0.002950,0.012085,0.023973,0.019841,0.039301,0.014281
FS,0.044592,0.026627,0.037975,0.038348,0.030211,0.037671,0.043651,0.026201,0.037766
ILB,0.052182,0.026627,0.047468,0.038348,0.057402,0.061644,0.051587,0.048035,0.048556
K,0.032258,0.000000,0.003165,0.000000,0.003021,0.013699,0.011905,0.013100,0.014599
LS,0.008539,0.000000,0.000000,0.000000,0.000000,0.003425,0.015873,0.000000,0.004443


As can be seen, there are a relatively high proportion of WRs, OTs, CBs, DEs, and DTs in the first round. In later rounds, More RBs and skill positions are taken. Let's now examine the proportions of rounds each position is typically drafted in.

In [15]:
round_info = pd.crosstab(index=df["Position"], columns=df["Round"], margins=True) 
round_info.div(round_info["All"], axis=0)

Round,0,1,2,3,4,5,6,7,All
Position,,,,,,,,,
C,0.375000,0.083333,0.093750,0.093750,0.135417,0.041667,0.093750,0.083333,1.0
CB,0.256560,0.128280,0.125364,0.145773,0.104956,0.116618,0.069971,0.052478,1.0
DE,0.260870,0.162055,0.118577,0.130435,0.098814,0.059289,0.083004,0.086957,1.0
DT,0.268000,0.136000,0.132000,0.148000,0.084000,0.076000,0.076000,0.080000,1.0
FB,0.422222,0.000000,0.000000,0.022222,0.088889,0.155556,0.111111,0.200000,1.0
FS,0.394958,0.075630,0.100840,0.109244,0.084034,0.092437,0.092437,0.050420,1.0
ILB,0.359477,0.058824,0.098039,0.084967,0.124183,0.117647,0.084967,0.071895,1.0
K,0.739130,0.000000,0.021739,0.000000,0.021739,0.086957,0.065217,0.065217,1.0
LS,0.642857,0.000000,0.000000,0.000000,0.000000,0.071429,0.285714,0.000000,1.0


At least 1/3 of players in most positions are undrafted; OTs have a significantly low amount undrafted, however. Most OTs are picked in the first round, with mostly decreasing numbers after that. We can see that positions such as kickers, punters, and fullbacks are primarily drafted in the late rounds.

# Simple Regression

For this, we will run a "one versus all" regression to predict what players will go in the first round.

In [16]:
df.loc[df.Round != 1, "Round"] = 0

In [17]:
all_X = df.drop(["Name", "Round", "Pick", "College", "Conference"], 1)
all_X = pd.get_dummies(all_X)

train_X = all_X[(all_X.Year != 2018)].drop("Year", 1)
test_X = all_X[all_X.Year == 2018].drop("Year", 1)
train_y = df[(df.Year != 2018)].Round
test_y = df[df.Year == 2018].Round

In [18]:
log = LogisticRegressionCV()
log.fit(train_X, train_y)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

We can also use XGBoost.

In [19]:
XGB = xgb.XGBClassifier(colsample_bytree=0.7,
 eta= 0.001,
 eval_metric= 'mae',
 max_depth= 6,
 min_child_weight= 15,
 objective= 'binary:logistic',
 subsample= 0.7)

In [20]:
XGB.fit(train_X, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, eta=0.001, eval_metric='mae', gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=15, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.7)

In [21]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [22]:
preds = 0.4 * clf.predict_proba(test_X) + 0.4 * XGB.predict_proba(test_X) + 0.2 * log.predict_proba(test_X)

In [23]:
count = 1
for i in pd.DataFrame(preds).sort_values(by=1, ascending=False).index:
    print(str(count) + " " + str(df[df.Year==2018].reset_index().at[i, "Name"]))
    count += 1

1 Kolton Miller
2 Mike McGlinchey
3 Saquon Barkley
4 Harold Landry
5 Terrell Edmunds
6 David Bright
7 Lamar Jackson
8 Justin Reid
9 Bradley Chubb
10 Derwin James
11 Martinas Rankin
12 Frank Ragnow
13 Josh Allen
14 Joseph Noteboom
15 D.J. Moore
16 Dane Cruikshank
17 Avonte Maddox
18 Sam Darnold
19 Harrison Phillips
20 Riley Ferguson
21 Jaire Alexander
22 Troy Apke
23 Minkah Fitzpatrick
24 Taven Bryan
25 Malik Jefferson
26 D.J. Chark
27 Kylie Fitts
28 Darius Phillips
29 Marquis Haynes
30 B.J. Hill
31 Josh Rosen
32 Godwin Igwebuike
33 Josh Sweat
34 Rod Taylor
35 Leighton Vander esch
36 Rick Leonard
37 Rashaad Penny
38 Quenton Nelson
39 Tremaine Edmunds
40 Courtland Sutton
41 Brian O'neill
42 Genard Avery
43 Sam Hubbard
44 Donte Jackson
45 Vita Vea
46 Scott Quessenberry
47 Quinton Flowers
48 Wyatt Teller
49 Jessie Bates
50 D.J. Reed
51 James Hearns
52 Anthony Averett
53 Baker Mayfield
54 Marcus Davenport
55 Matthew Thomas
56 Denzel Ward
57 Nyheim Hines
58 Deadrin Senat
59 Logan Woodside
60